In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import warnings
import matplotlib as mpl
import numpy as np
import pandas as pd  # Basic library for all of our dataset operations
import statsmodels as sm
import tensorflow as tf
from matplotlib import pyplot as plt
from pylab import rcParams
from sklearn.linear_model import LinearRegression
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa import api as smt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from math import sqrt
from numpy import split, array
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, RepeatVector, TimeDistributed, ConvLSTM2D
from sklearn.model_selection import GridSearchCV
import optuna
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Dense, Flatten, Dropout
from keras.optimizers import Adam
import numpy as np

In [ ]:
import pandas as pd

# Load the dataset
path = '/content/drive/MyDrive/Data_Science/01_Projects/04_Pakistan_Energy_Demand/Data/pak_load_till_2023_output.csv'
dataset = pd.read_csv(path)

# Convert the 'datetime' columns to a single datetime column
dataset['datetime'] = pd.to_datetime(dataset[['YEAR', 'MONTH', 'DAY', 'Hour']])

# Set 'datetime' as the index
dataset.set_index('datetime', inplace=True)

# Drop unnecessary columns
dataset.drop(['YEAR', 'MONTH', 'DAY', 'Hour'], axis=1, inplace=True)

# Extract additional date-related features
dataset['year'] = dataset.index.year
dataset['month'] = dataset.index.month
dataset['dayofweek'] = dataset.index.dayofweek
dataset['hour'] = dataset.index.hour

# Display the first row of the modified dataset
print(dataset.head(10))


                         Demand  year  month  dayofweek  hour
datetime                                                     
2019-07-01 01:00:00  19615.2656  2019      7          0     1
2019-07-01 02:00:00  17842.2494  2019      7          0     2
2019-07-01 03:00:00  17663.2494  2019      7          0     3
2019-07-01 04:00:00  18715.0406  2019      7          0     4
2019-07-01 05:00:00  18839.8286  2019      7          0     5
2019-07-01 06:00:00  18170.1786  2019      7          0     6
2019-07-01 07:00:00  17795.2036  2019      7          0     7
2019-07-01 08:00:00  18957.4286  2019      7          0     8
2019-07-01 09:00:00  19725.1536  2019      7          0     9
2019-07-01 10:00:00  19885.4036  2019      7          0    10


In [ ]:
dataset.head()
# Assuming you have a DataFrame df with your dataset
dataset = dataset[['year', 'month', 'dayofweek', 'hour', 'Demand']]
dataset.head(1)


,year,month,dayofweek,hour,Demand
datetime,,,,,
2019-07-01 01:00:00,2019,7,0,1,19615.2656


In [ ]:
data = dataset

In [ ]:
# Resample the dataset at a daily frequency and calculate the mean
newDataSet = data.resample("D").mean()
# Omit one row at the end of the resampled dataset
# 1646 -1 =1645/7 235 weeks
# 235 * 80 = 188 for train
# 235 - 188 = 47 for test
newDataSet = newDataSet.iloc[:-1]
# Display the first few rows of the resampled dataset
print(newDataSet.head())
print(newDataSet.shape)

              year  month  dayofweek  hour        Demand
datetime                                                
2019-07-01  2019.0    7.0        0.0  12.0  19309.086209
2019-07-02  2019.0    7.0        1.0  11.5  19667.606325
2019-07-03  2019.0    7.0        2.0  11.5  20422.855358
2019-07-04  2019.0    7.0        3.0  11.5  19702.835742
2019-07-05  2019.0    7.0        4.0  11.5  18838.397733
(1645, 5)


In [ ]:
y = newDataSet["Demand"]
print(y[0])
y.shape

19309.08620869565


(1645,)

In [ ]:
# Normalize data before model fitting
# it will boost the performance( in neural networks) + transform
from sklearn.preprocessing import MinMaxScaler
# scale of the output and input inthe range 0-1 to match the scale of the layer of LSTM
scaler = MinMaxScaler(feature_range = (0,1))
# reshape: convert the univariate 1D array into 2D
y = scaler.fit_transform(np.array(y).reshape(-1,1))
print("Normalizing data before model fitting")
print(y[:10])

Normalizing data before model fitting
[[0.91806392]
 [0.94443899]
 [1.        ]
 [0.94703069]
 [0.88343703]
 [0.81160418]
 [0.89298101]
 [0.98384646]
 [0.9446605 ]
 [0.95068641]]


In [ ]:
training_size = int(len(y)*0.80)
test_size = len(y)- training_size
val_size = int(training_size*0.20)
train_data , test_data , val_data = y[0:training_size-val_size,:] , y[training_size:len(y),:1], y[len(y)-test_size-val_size:len(y)-test_size,:1]


In [ ]:
# building input variable
def create_dataset(dataset, time_step = 1):
  dataX, dataY = [] , []
  for i in range(len(dataset)-time_step-1):
    a = dataset[i:(i+time_step),0]
    dataX.append(a)
    dataY.append(dataset[i + time_step,0])
  return np.array(dataX), np.array(dataY)


In [ ]:
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)
X_val, yval = create_dataset(val_data, time_step)


In [ ]:

# reshape train and input-output pairs
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1],1)

print("X_train shape: ", X_train.shape)
print("X_test shape: ",X_test.shape)
print("X_val shape: ",X_val.shape)


X_train shape:  (952, 100, 1)
X_test shape:  (228, 100, 1)
X_val shape:  (162, 100, 1)


In [ ]:
! pip install optuna

In [ ]:
import optuna
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
import numpy as np

# Define the objective function for Optuna
def objective(trial):
    n_lstm_layers = trial.suggest_int('n_lstm_layers', 1, 4)
    lstm_units = trial.suggest_int('lstm_units', 32, 128)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)

    model = Sequential()
    model.add(LSTM(units=lstm_units, return_sequences=True, input_shape=(time_step, 1)))
    model.add(Dropout(dropout_rate))

    for _ in range(n_lstm_layers - 1):
        model.add(LSTM(units=lstm_units, return_sequences=True))
        model.add(Dropout(dropout_rate))

    model.add(LSTM(units=lstm_units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')



    history = model.fit(X_train, y_train, validation_data=(X_val, yval), epochs=trial.suggest_int('epochs', 10, 50), batch_size=trial.suggest_int('batch_size', 16, 64), verbose=2)

    val_loss = model.evaluate(X_val, yval, verbose=0)
    return val_loss

# Create an Optuna study
study = optuna.create_study(direction='minimize')

# Optimize hyperparameters
study.optimize(objective, n_trials=10)

# Get the best hyperparameters
best_params = study.best_params

# Build the final model with the best hyperparameters
best_model = Sequential()
best_model.add(LSTM(units=best_params['lstm_units'], return_sequences=True, input_shape=(time_step, 1)))
best_model.add(Dropout(best_params['dropout_rate']))

for _ in range(best_params['n_lstm_layers'] - 1):
    best_model.add(LSTM(units=best_params['lstm_units'], return_sequences=True))
    best_model.add(Dropout(best_params['dropout_rate']))

best_model.add(LSTM(units=best_params['lstm_units']))
best_model.add(Dropout(best_params['dropout_rate']))
best_model.add(Dense(units=1))

optimizer = Adam(learning_rate=best_params['learning_rate'])
best_model.compile(optimizer=optimizer, loss='mean_squared_error')



In [ ]:
best_model.summary()

In [ ]:
history = model.fit(X_train, y_train, validation_data = (X_val,yval), verbose = 1,epochs = 60 ,batch_size = 20)


In [ ]:
model.save('/content/drive/MyDrive/Data_Science/01_Projects/04_Pakistan_Energy_Demand/Trained_models/Stacked_LSTM')

In [ ]:

plt.figure(figsize=(5,5))
plt.plot(history.history['loss']) # tb
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
